# Business Data Analytics - Challenge

---

## Package Import

In [1]:
import pandas as pd
import numpy as np
import re
import os

---

In [2]:
def _create_datetime(row):
    date = row.Datum.strftime("%Y-%m-%d") + " " + row.Uhrzeit
    return date

In [3]:
def prepare_stromfluesse(path_to_data_storage):
    
    dateparse = lambda x: pd.datetime.strptime(x, '%d.%m.%Y')
    numberparse = lambda x: pd.np.float(x.replace(".", "").replace(",",".")) if x!="-" else np.nan
    convert_thousand = { num:numberparse   for num in np.arange(2,22)}
    
    import_files = os.listdir(path_to_data_storage)

    for idx , file in enumerate(import_files):
        print("Import File: {} ".format(file))
        PATH = path_to_data_storage + file
        if idx>0:
            df2 = pd.read_csv(PATH,
                sep=r";",
                decimal=r",",
                thousands=r".",
                converters = convert_thousand,
                parse_dates=['Datum'],
                date_parser = dateparse )
            df = df.append(df2)
        else:
            df = pd.read_csv(PATH,
                sep=r";",
                decimal=r",",
                thousands=r".",
                converters = convert_thousand,
                parse_dates=['Datum'],
                date_parser = dateparse)
    
    df['Date'] = df.apply(lambda row: _create_datetime(row), axis=1) 
    df['Date'] = pd.to_datetime(df.Date,format="%Y-%m-%d %H:%M")
    df = df.sort_values('Date').reset_index(drop=True)

    cols = list(df)
    cols.insert(0, cols.pop(cols.index('Date')))
    df = df.loc[:, cols]
    df.drop(['Uhrzeit'],axis=1,inplace=True)
    
    type_pattern = r"\((.*?)\)"
    country_pattern = r"(.*?) "
    countries = {
    'Niederlande':'NL',
    'Schweiz': 'CHE',
    'Dänemark' : 'DNK',
    'Tschechien' : 'CZE',
    'Luxemburg' : 'LUX',
    'Schweden' : 'SWE',
    'Österreich' : 'AUT',
    'Frankreich' : 'FRA',
    'Polen' : 'PL'
    } 

    types = {
        'Import' : 'IM',
        'Export' : 'EX'
    }

    new_columns = [countries.get(re.search(country_pattern,col).group(1)) + 
     "_" + 
     types.get(re.search(type_pattern,col).group(1))
     for col in df.columns[3::]]
    new_columns.insert(0,'Date')
    new_columns.insert(1,'Tag')
    new_columns.insert(2,'NX')
    df.columns = new_columns
    
    df['NX'] = df.loc[:, 'NL_EX':'PL_IM'].sum(axis=1)
    
    df = df.fillna(0)
    
    for key,value in countries.items():
        expo = value + "_EX"
        impo = value + "_IM"
        df[value] = df[expo] + df[impo]
        df.drop([expo,impo],axis=1,inplace=True)

    return df

In [4]:
def prepare_strompreise(path_to_data_storage,aggregate=True):
    import_files = os.listdir(path_to_data_storage)

    for idx , file in enumerate(import_files):
        print("Import File: {} ".format(file))
        PATH = path_to_data_storage + file
        if idx>0:
            df2 = pd.read_csv(PATH,
                sep=r";",
                decimal=r",",
                thousands=r".",
                converters = convert_thousand,
                parse_dates=['Datum'],
                date_parser = dateparse )
            df_price = df_price.append(df2)
        else:
            df_price = pd.read_csv(PATH,
                sep=r";",
                decimal=r",",
                thousands=r".",
                converters = convert_thousand,
                parse_dates=['Datum'],
                date_parser = dateparse)
    
    df_price['Date'] = df_price.apply(lambda row: _create_datetime(row), axis=1) 
    df_price['Date'] = pd.to_datetime(df_price.Date,format="%Y-%m-%d %H:%M")
    df_price = df_price.sort_values('Date').reset_index(drop=True)

    cols = list(df_price)
    cols.insert(0, cols.pop(cols.index('Date')))
    df_price = df_price.loc[:, cols]
    df_price.drop(['Uhrzeit'],axis=1,inplace=True)
    
    delete_currency = r"(.*?)\["
    new_columns = ["price_"+ re.search(delete_currency,col).group(1).lower()
     for col in df_price.columns[2::]]
    new_columns.insert(0,'Date')
    new_columns.insert(1,'Tag')
    df_price.columns = new_columns
    
    df_price.fillna(df_price.mean(),inplace=True)
    
    df_price_aggregated = pd.DataFrame(columns=df_price.columns[2::])

    ind = 0 
    for name,df in df_price.groupby('Tag'):
        mean_price_day = df.iloc[:,2::].mean()
        df_price_aggregated = df_price_aggregated.append(mean_price_day,ignore_index=True)

    df_price_aggregated.columns = ["daily_" + col for col in df_price_aggregated.columns]
    
    df_price_aggregated['Tag'] = df_price.Tag.unique()
    
    cols = list(df_price_aggregated)
    cols.insert(0, cols.pop(cols.index('Tag')))
    df_price_aggregated = df_price_aggregated.loc[:, cols]
    
    if aggregate == True:
        return df_price_aggregated
    else:
        return df_price

In [5]:
def prepare_konsum_ger(path_to_data_storage):
    import_files = os.listdir(path_to_data_storage)

    for idx , file in enumerate(import_files):
        print("Import File: {} ".format(file))
        PATH = path_to_data_storage + file

        if idx>0:
            df2 = pd.read_csv(PATH,
                sep=r";",
                decimal=r",",
                thousands=r".",
                converters = convert_thousand,
                parse_dates=['Datum'],
                date_parser = dateparse )
            df_consumption = df_consumption.append(df2)
        else:
            df_consumption = pd.read_csv(PATH,
                sep=r";",
                decimal=r",",
                thousands=r".",
                converters = convert_thousand,
                parse_dates=['Datum'],
                date_parser = dateparse)

    df_consumption = df_consumption.groupby("Datum").sum()
    df_consumption.columns = ["daily_consumption_ger"]

    df_consumption.dropna(inplace=True)
    df_consumption['Tag'] = df_consumption.index
    
    cols = list(df_consumption)
    cols.insert(0, cols.pop(cols.index('Tag')))
    df_consumption = df_consumption.loc[:, cols]

    df_consumption.reset_index(drop=True,inplace=True)
    
    return df_consumption

---

---

## Data Import

In [6]:
dateparse = lambda x: pd.datetime.strptime(x, '%d.%m.%Y')
numberparse = lambda x: pd.np.float(x.replace(".", "").replace(",",".")) if x!="-" else np.nan
convert_thousand = { num:numberparse   for num in np.arange(2,22)}

In [7]:
import_files = os.listdir("data/Stromfluss/")

for idx , file in enumerate(import_files):
    print("Import File: {} ".format(file))
    PATH = "data/Stromfluss/" + file
    if idx>0:
        df2 = pd.read_csv(PATH,
            sep=r";",
            decimal=r",",
            thousands=r".",
            converters = convert_thousand,
            parse_dates=['Datum'],
            date_parser = dateparse )
        df = df.append(df2)
    else:
        df = pd.read_csv(PATH,
            sep=r";",
            decimal=r",",
            thousands=r".",
            converters = convert_thousand,
            parse_dates=['Datum'],
            date_parser = dateparse)
del df2

Import File: DE_Physikalischer_Stromfluss_201506010000_201706012359_1.csv 
Import File: DE_Physikalischer_Stromfluss_201706020000_201906022359_1.csv 


---

## Preprocessing

#### Time

In [8]:
def _create_datetime(row):
    date = row.Datum.strftime("%Y-%m-%d") + " " + row.Uhrzeit
    return date

In [9]:
df['Date'] = df.apply(lambda row: _create_datetime(row), axis=1) 
df['Date'] = pd.to_datetime(df.Date,format="%Y-%m-%d %H:%M")
df = df.sort_values('Date').reset_index(drop=True)

cols = list(df)
cols.insert(0, cols.pop(cols.index('Date')))
df = df.loc[:, cols]
df.drop(['Uhrzeit'],axis=1,inplace=True)

#### Columns

In [7]:
type_pattern = r"\((.*?)\)"
country_pattern = r"(.*?) "

In [8]:
countries = {
    'Niederlande':'NL',
    'Schweiz': 'CHE',
    'Dänemark' : 'DNK',
    'Tschechien' : 'CZE',
    'Luxemburg' : 'LUX',
    'Schweden' : 'SWE',
    'Österreich' : 'AUT',
    'Frankreich' : 'FRA',
    'Polen' : 'PL'
} 

types = {
    'Import' : 'IM',
    'Export' : 'EX'
}

new_columns = [countries.get(re.search(country_pattern,col).group(1)) + 
 "_" + 
 types.get(re.search(type_pattern,col).group(1))
 for col in df.columns[3::]]
new_columns.insert(0,'Date')
new_columns.insert(1,'Tag')
new_columns.insert(2,'NX')

df.columns = new_columns

#### NX

In [11]:
df['NX'] = df.loc[:, 'NL_EX':'PL_IM'].sum(axis=1)

#### Fill Nones

In [12]:
df = df.fillna(0)

### Aggregate to NX

In [13]:
for key,value in countries.items():
    expo = value + "_EX"
    impo = value + "_IM"
    df[value] = df[expo] + df[impo]
    df.drop([expo,impo],axis=1,inplace=True)


In [14]:
df.head()

,Date,Tag,NX,NL,CHE,DNK,CZE,LUX,SWE,AUT,FRA,PL
0,2015-06-01 00:00:00,2015-06-01,7279.0,3932.0,145.0,-272.0,554.0,0.0,1.0,1394.0,246.0,1279.0
1,2015-06-01 01:00:00,2015-06-01,7167.0,4014.0,261.0,-141.0,469.0,0.0,1.0,1355.0,66.0,1142.0
2,2015-06-01 02:00:00,2015-06-01,6667.0,3864.0,103.0,-326.0,422.0,0.0,1.0,1574.0,-105.0,1134.0
3,2015-06-01 03:00:00,2015-06-01,6413.0,4242.0,42.0,-262.0,235.0,0.0,1.0,1271.0,-206.0,1090.0
4,2015-06-01 04:00:00,2015-06-01,5347.0,4292.0,-127.0,-582.0,161.0,0.0,1.0,1068.0,-549.0,1083.0


---

# Strompreise

In [22]:
df_price = prepare_strompreise("data/Strompreise/")

Import File: DE_Großhandelspreise_201706010000_201906012359_1.csv 
Import File: DE_Großhandelspreise_201506010000_201706012359_1.csv 


In [23]:
import_files = os.listdir("data/Strompreise/")

for idx , file in enumerate(import_files):
    print("Import File: {} ".format(file))
    PATH = "data/Strompreise/" + file
    if idx>0:
        df2 = pd.read_csv(PATH,
            sep=r";",
            decimal=r",",
            thousands=r".",
            converters = convert_thousand,
            parse_dates=['Datum'],
            date_parser = dateparse )
        df_price = df_price.append(df2)
    else:
        df_price = pd.read_csv(PATH,
            sep=r";",
            decimal=r",",
            thousands=r".",
            converters = convert_thousand,
            parse_dates=['Datum'],
            date_parser = dateparse)
del df2

Import File: DE_Großhandelspreise_201706010000_201906012359_1.csv 
Import File: DE_Großhandelspreise_201506010000_201706012359_1.csv 


In [24]:
df_price['Date'] = df_price.apply(lambda row: _create_datetime(row), axis=1) 
df_price['Date'] = pd.to_datetime(df_price.Date,format="%Y-%m-%d %H:%M")
df_price = df_price.sort_values('Date').reset_index(drop=True)

cols = list(df_price)
cols.insert(0, cols.pop(cols.index('Date')))
df_price = df_price.loc[:, cols]
df_price.drop(['Uhrzeit'],axis=1,inplace=True)

In [25]:
delete_currency = r"(.*?)\["

new_columns = ["price_"+ re.search(delete_currency,col).group(1).lower()
 for col in df_price.columns[2::]]
new_columns.insert(0,'Date')
new_columns.insert(1,'Tag')

df_price.columns = new_columns

In [26]:
df_price.fillna(df_price.mean(),inplace=True)

In [27]:
df_price_aggregated = pd.DataFrame(columns=df_price.columns[2::])

ind = 0 
for name,df in df_price.groupby('Tag'):
    mean_price_day = df.iloc[:,2::].mean()
    df_price_aggregated = df_price_aggregated.append(mean_price_day,ignore_index=True)

df_price_aggregated.columns = ["daily_" + col for col in df_price_aggregated.columns]

In [28]:
df_price_aggregated['Tag'] = df_price.Tag.unique()

In [33]:
cols = list(df_price_aggregated)
cols.insert(0, cols.pop(cols.index('Tag')))
df_price_aggregated = df_price_aggregated.loc[:, cols]

In [34]:
df_price_aggregated.head()

,Tag,daily_price_deutschland/luxemburg,daily_price_dänemark 1,daily_price_dänemark 2,daily_price_frankreich,daily_price_italien (nord),daily_price_niederlande,daily_price_polen,daily_price_schweden 4,daily_price_schweiz,daily_price_slowenien,daily_price_tschechien,daily_price_ungarn,daily_price_österreich,daily_price_deutschland/österreich/luxemburg
0,2015-06-01,44.390017,20.788750,30.169583,30.997500,49.521667,42.491250,39.270939,30.169583,32.077083,37.734167,32.905833,39.271250,48.781522,31.438333
1,2015-06-02,44.390017,16.130417,16.762500,24.302083,43.613333,33.039583,40.454272,16.762500,24.202500,28.686667,25.392500,41.874583,48.781522,22.030000
2,2015-06-03,44.390017,15.313750,15.353333,30.330000,52.009167,37.045833,40.272606,15.353333,26.726667,43.868750,27.514167,40.420833,48.781522,27.750000
3,2015-06-04,44.390017,22.298750,23.852917,31.705833,54.611667,46.650417,34.739724,23.462917,29.455833,37.598333,29.890417,34.675417,48.781522,28.929167
4,2015-06-05,44.390017,17.689583,21.050833,39.059583,51.943333,43.321250,43.065957,21.050833,29.855000,33.243333,30.061667,34.020417,48.781522,29.611667


---

# Realisierter Stromverbrauch

In [43]:
import_files = os.listdir("data/Stromverbrauch_real/")

for idx , file in enumerate(import_files):
    print("Import File: {} ".format(file))
    PATH = "data/Stromverbrauch_real/" + file
    
    if idx>0:
        df2 = pd.read_csv(PATH,
            sep=r";",
            decimal=r",",
            thousands=r".",
            converters = convert_thousand,
            parse_dates=['Datum'],
            date_parser = dateparse )
        df_consumption = df_consumption.append(df2)
    else:
        df_consumption = pd.read_csv(PATH,
            sep=r";",
            decimal=r",",
            thousands=r".",
            converters = convert_thousand,
            parse_dates=['Datum'],
            date_parser = dateparse)

del df2

Import File: DE_Realisierter Stromverbrauch_201506010000_201706012345_1.csv 
Import File: DE_Realisierter Stromverbrauch_201706010000_201906012345_1.csv 


In [52]:
df_consumption = df_consumption.groupby("Datum").sum()
df_consumption.columns = ["daily_consumption_ger"]

df_consumption.dropna(inplace=True)
df_consumption['Tag'] = df_consumption.index

cols = list(df_consumption)
cols.insert(0, cols.pop(cols.index('Tag')))
df_consumption = df_consumption.loc[:, cols]

df_consumption.reset_index(drop=True,inplace=True)

In [53]:
df_consumption.head()

,Tag,daily_consumption_ger
0,2015-06-01,1284069.00
1,2015-06-02,1365352.75
2,2015-06-03,1339142.50
3,2015-06-04,1104016.75
4,2015-06-05,1212480.00


---

# Merge Preis und Tagesverbrauch

In [57]:
df_stromfluesse = prepare_stromfluesse("data/Stromfluss//")
df_price = prepare_strompreise("data/Strompreise/")
df_consumption = prepare_konsum_ger("data/Stromverbrauch_real/")

Import File: DE_Physikalischer_Stromfluss_201506010000_201706012359_1.csv 
Import File: DE_Physikalischer_Stromfluss_201706020000_201906022359_1.csv 
Import File: DE_Großhandelspreise_201706010000_201906012359_1.csv 
Import File: DE_Großhandelspreise_201506010000_201706012359_1.csv 
Import File: DE_Realisierter Stromverbrauch_201506010000_201706012345_1.csv 
Import File: DE_Realisierter Stromverbrauch_201706010000_201906012345_1.csv 


---

In [75]:
def enrich_daily_information(df_price,df_consumption):
    
    df_daily_information = df_consumption.merge(df_price_aggregated,on='Tag')
    cols = ["prev_day" + col[5::] for col in df_daily_information.columns[1::]]
    cols.insert(0,'Tag')
    df_daily_information.columns = cols
    # Shift
    day_shift = 1 
    df_daily_information.iloc[:,1::] = df_daily_information.iloc[:,1::].shift(day_shift)
    df_daily_information.dropna(inplace=True)
    
    # Zusätliche Tagesinformationen
    df_daily_information['time_diff_days'] = (df_daily_information['Tag'] - pd.Timestamp(df_daily_information['Tag'].min())).dt.days

    df_daily_information['day_of_week'] = df_daily_information['Tag'].dt.day_name()
    df_daily_information = pd.concat([df_daily_information, pd.get_dummies(df_daily_information['day_of_week'])], axis=1)      
    df_daily_information.drop(['day_of_week'], inplace=True, axis=1)

    df_daily_information['month'] = df_daily_information['Tag'].dt.month
    df_daily_information = pd.concat([df_daily_information, pd.get_dummies(df_daily_information['month'])], axis=1)      
    df_daily_information.drop(['month'], inplace=True, axis=1)
    
    return df_daily_information

In [76]:
df_daily_information = enrich_daily_information(df_price,df_consumption)

In [78]:
df_daily_information.head()

,Tag,daily_consumption_ger,daily_price_deutschland/luxemburg,daily_price_dänemark 1,daily_price_dänemark 2,daily_price_frankreich,daily_price_italien (nord),daily_price_niederlande,daily_price_polen,daily_price_schweden 4,...,3,4,5,6,7,8,9,10,11,12
1,2015-06-02,1284069.00,44.390017,20.788750,30.169583,30.997500,49.521667,42.491250,39.270939,30.169583,...,0,0,0,1,0,0,0,0,0,0
2,2015-06-03,1365352.75,44.390017,16.130417,16.762500,24.302083,43.613333,33.039583,40.454272,16.762500,...,0,0,0,1,0,0,0,0,0,0
3,2015-06-04,1339142.50,44.390017,15.313750,15.353333,30.330000,52.009167,37.045833,40.272606,15.353333,...,0,0,0,1,0,0,0,0,0,0
4,2015-06-05,1104016.75,44.390017,22.298750,23.852917,31.705833,54.611667,46.650417,34.739724,23.462917,...,0,0,0,1,0,0,0,0,0,0
5,2015-06-06,1212480.00,44.390017,17.689583,21.050833,39.059583,51.943333,43.321250,43.065957,21.050833,...,0,0,0,1,0,0,0,0,0,0


## Daily Informations

In [71]:
df_daily_information = df_consumption.merge(df_price_aggregated,on='Tag')


In [ ]:
cols = ["prev_day" + col[5::] for col in df_daily_information.columns[1::]]
cols.insert(0,'Tag')
df_daily_information.columns = cols

In [221]:
# Shift
day_shift = 1 
df_daily_information.iloc[:,1::] = df_daily_information.iloc[:,1::].shift(day_shift)
df_daily_information.dropna(inplace=True)

In [74]:
# Zusätliche Tagesinformationen
df_daily_information['time_diff_days'] = (df_daily_information['Tag'] - pd.Timestamp(df_daily_information['Tag'].min())).dt.days

df_daily_information['day_of_week'] = df_daily_information['Tag'].dt.day_name()
df_daily_information = pd.concat([df_daily_information, pd.get_dummies(df_daily_information['day_of_week'])], axis=1)      
df_daily_information.drop(['day_of_week'], inplace=True, axis=1)

df_daily_information['month'] = df_daily_information['Tag'].dt.month
df_daily_information = pd.concat([df_daily_information, pd.get_dummies(df_daily_information['month'])], axis=1)      
df_daily_information.drop(['month'], inplace=True, axis=1)

---

---

# Merge Tagesinformationen mit NX

In [239]:
df = df.merge(df_aggregated,on="Tag")

In [241]:
df.to_csv("data/data_preprocessed.csv",sep=";")